In [1]:
#location of the dataset
#DATA_SRC='/home/beangoben/ml_data/mpd.v1/data'
DATA_SRC='/home/beangoben/data/mpd.v1/data'
#DATA_SRC='../data/raw/5k_subset'
# add src scripts to utilize
import sys
sys.path.append('../src')
sys.path.append('../src/data/')
sys.path.append('../src/models/')
sys.path.append('../src/features/')
sys.path.append('../src/visualization/')
# will reload any library
%load_ext autoreload
%autoreload 2

In [2]:
# sportify specific
from data.metrics_track import r_precision, cosine_sim_closest, cosine_sim_top, NDCG
from visualization.plot_utils import write_latex_table
# general
import pandas as pd
import numpy as np
from tqdm import tqdm
from features.audio_features import AudioFeatures
from features.tracks_info import TrackInfo
from collections import OrderedDict

In [3]:
k_range = [1, 5, 10, 25, 100]
max_picks = 500

tracks = TrackInfo(subset='5k-')
# load playlist
pid = 194
pl_turi,pl_ints,pl_auri = tracks.get_playlist(pid)
n_songs = len(pl_uri)

NameError: name 'pl_uri' is not defined

In [4]:
# audio features
af = AudioFeatures(subset='5k-')
X = af.transform()
n_cols = X.shape[1]

Index(['acousticness', 'danceability', 'duration_ms', 'energy', 'liveness',
       'loudness', 'speechiness', 'tempo', 'valence'],
      dtype='object')


In [25]:
xp = af.subset(pl_uri)

In [26]:
suggest_df = pd.DataFrame()
n_top = 10
x_pred = OrderedDict()
stats = []
print('Test on playlist {:d}'.format(pid))
for k in k_range:
    x_pred[k] = []
    k_picks = int(max_picks / k)
    for i in range(k):
        x_query = xp[i - 1, :]
        ind, sim = cosine_sim_closest(X, x_query, k_picks)
        x_pred[k] = x_pred[k] + list(ind)
    subset_ids = p_ids[k:]
    result = OrderedDict()
    result['k'] = k
    result['r precision'] = r_precision(subset_ids, x_pred[k])
    result['NDGC'] = NDCG(subset_ids, x_pred[k])
    stats.append(result)

sdf = pd.DataFrame(stats)
print(sdf)

write_latex_table(sdf, 'metrics', adir='.', render=True)

Test on playlist 194
     k  r precision  NDGC
0    1          0.0   0.0
1    5          0.0   0.0
2   10          0.0   0.0
3   25          0.0   0.0
4  100          0.0   0.0
